In [53]:
# packages we need:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
# For data manipulation
from functools import partial

# These two are for cool progress bars
from tqdm import tqdm
import urllib.request
tqdm = partial(tqdm, position=0, leave=True)
import re

# For our model(s)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import PoissonRegressor

In [ ]:
matches_final = pd.read_csv('matches_final.csv')

In [ ]:
team_names = matches_final['Opponent'].unique()

cleaned_team_names = [team.split(' ', 1)[1] for team in team_names]

name_mapping = dict(zip(team_names, cleaned_team_names))

matches_final['Opponent'] = matches_final['Opponent'].replace(name_mapping)

In [ ]:
# Create empty columns for HomeTeam and AwayTeam
matches_final['HomeTeam'] = ''
matches_final['AwayTeam'] = ''

# Iterate through each row and set HomeTeam and AwayTeam based on the value in the Venue column
for index, row in matches_final.iterrows():
    if row['Venue'] == 'Home':
        matches_final.at[index, 'HomeTeam'] = row['Team']
        matches_final.at[index, 'AwayTeam'] = row['Opponent']
    elif row['Venue'] == 'Away':
        matches_final.at[index, 'HomeTeam'] = row['Opponent']
        matches_final.at[index, 'AwayTeam'] = row['Team']

In [ ]:
matches_final = matches_final[matches_final['Venue'] != 'Neutral']

In [ ]:
# Define the mapping of old team names to new team names
# Mapping dictionary for old and new team names
team_name_mapping = {
    'Tottenham': 'Tottenham Hotspur',
    'Manchester Utd': 'Manchester United',
    'Paris S-G': 'Paris Saint Germain',
    'Atlético Madrid': 'Atletico Madrid',
    'Inter': 'Internazionale',
    'PSV Eindhoven': 'PSV'
}

# Replace old team names with new ones in both HomeTeam and AwayTeam columns
matches_final['HomeTeam'] = matches_final['HomeTeam'].replace(team_name_mapping)
matches_final['AwayTeam'] = matches_final['AwayTeam'].replace(team_name_mapping)

<ipython-input-11-1b7da6e5f742>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_final['HomeTeam'] = matches_final['HomeTeam'].replace(team_name_mapping)
<ipython-input-11-1b7da6e5f742>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_final['AwayTeam'] = matches_final['AwayTeam'].replace(team_name_mapping)


In [ ]:
matches_final = matches_final.drop_duplicates(subset=['Date', 'HomeTeam', 'AwayTeam'])

In [ ]:
# Assuming df is your DataFrame

# Identify rows where 'Venue' is 'Away'
away_rows = matches_final['Venue'] == 'Away'

# Swap the values of 'GF' and 'GA' for those rows
matches_final.loc[away_rows, ['GF', 'GA']] = matches_final.loc[away_rows, ['GA', 'GF']].values



In [ ]:
import numpy as np

# Define conditions and corresponding values for 'Result'
conditions = [
    matches_final['GF'] > matches_final['GA'],
    matches_final['GF'] < matches_final['GA'],
    matches_final['GF'] == matches_final['GA']
]
values = ['H', 'A', 'D']

# Set 'Result' based on conditions
matches_final['Result'] = np.select(conditions, values)



In [ ]:
results_data = matches_final[['Date','HomeTeam','AwayTeam','GF', 'GA', 'Result']]

In [ ]:
results_data.columns = ['Date','HomeTeam','AwayTeam','FTHG', 'FTAG', 'FTR']

In [ ]:
import re

# Function to remove parentheses and their contents
def remove_parentheses(value):
    if isinstance(value, str):
        return re.sub(r'\([^)]*\)', '', value)
    else:
        return value

# Apply the function to the 'GF' and 'GA' columns
results_data['FTHG'] = results_data['FTHG'].apply(remove_parentheses)
results_data['FTAG'] = results_data['FTAG'].apply(remove_parentheses)



<ipython-input-27-f2f9a2688817>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_data['FTHG'] = results_data['FTHG'].apply(remove_parentheses)
<ipython-input-27-f2f9a2688817>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_data['FTAG'] = results_data['FTAG'].apply(remove_parentheses)


In [ ]:
results_data['FTHG'] = pd.to_numeric(results_data['FTHG'])
results_data['FTAG'] = pd.to_numeric(results_data['FTAG'])

<ipython-input-28-31047e1c4b06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_data['FTHG'] = pd.to_numeric(results_data['FTHG'])
<ipython-input-28-31047e1c4b06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_data['FTAG'] = pd.to_numeric(results_data['FTAG'])


# How to calculate ELO scores

In [3]:
df = pd.read_csv('updated_dataset_with_elo.csv')

In [4]:
results_data = df.drop(columns = ['Unnamed: 0','home_elo','away_elo'])

In [5]:
results_data = results_data.sort_values(by= 'Date')

In [6]:
results_data

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
937,1993-09-15,Honvéd,Manchester United,2.0,3.0,A
2016,1993-09-15,Aarau,Milan,0.0,1.0,A
1605,1993-09-15,Dynamo Kyiv,Barcelona,3.0,1.0,H
938,1993-09-29,Manchester United,Honvéd,2.0,1.0,H
2017,1993-09-29,Milan,Aarau,0.0,0.0,D
...,...,...,...,...,...,...
1623,2023-04-19,Internazionale,Benfica,3.0,3.0,D
10,2023-05-09,Real Madrid,Manchester City,1.0,1.0,D
1624,2023-05-10,Milan,Internazionale,0.0,2.0,A
1625,2023-05-16,Internazionale,Milan,1.0,0.0,H


In [7]:
def create_league_table(data):

    unique_team_names = data['HomeTeam'].unique()

    league_table = pd.DataFrame()

    league_table['team'] = unique_team_names
    league_table['ranking'] = 0
    league_table['points'] = 0
    league_table['w'] = 0
    league_table['d'] = 0
    league_table['l'] = 0
    league_table['goals_for'] = 0
    league_table['goals_against'] = 0
    league_table['goal_difference'] = 0
    league_table['matched_played'] = 0
    league_table['last_date'] = 0

    # if there is colname 'Date', then we can use it to update 'last_date' column:
    if 'Date' in data.columns:
        league_table['last_date'] = min(data['Date'])
    else:
        league_table['last_date'] = 0

    return league_table

In [8]:
table_data = create_league_table(results_data)

In [9]:
table_data

,team,ranking,points,w,d,l,goals_for,goals_against,goal_difference,matched_played,last_date
0,Honvéd,0,0,0,0,0,0,0,0,0,1993-09-15
1,Aarau,0,0,0,0,0,0,0,0,0,1993-09-15
2,Dynamo Kyiv,0,0,0,0,0,0,0,0,0,1993-09-15
3,Manchester United,0,0,0,0,0,0,0,0,0,1993-09-15
4,Milan,0,0,0,0,0,0,0,0,0,1993-09-15
...,...,...,...,...,...,...,...,...,...,...,...
154,Başakşehir,0,0,0,0,0,0,0,0,0,1993-09-15
155,Rennes,0,0,0,0,0,0,0,0,0,1993-09-15
156,Midtjylland,0,0,0,0,0,0,0,0,0,1993-09-15
157,Sheriff Tiraspol,0,0,0,0,0,0,0,0,0,1993-09-15


In [10]:
def update_league_table(league_table, results_table):

    league_table_all = league_table.copy()
    league_table_new = league_table.copy()

    for index, row in results_table.iterrows():
        # get team names
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        # get match results
        home_team_goals = row['FTHG']
        away_team_goals = row['FTAG']
        full_time_result = row['FTR']
        # get date of match
        last_date = row['Date']

        # update 'matched_played' column:
        league_table_new.loc[league_table_new['team'] == home_team, 'matched_played'] += 1
        league_table_new.loc[league_table_new['team'] == away_team, 'matched_played'] += 1

        # update 'goals_for' and 'goals_against' columns:
        league_table_new.loc[league_table_new['team'] == home_team, 'goals_for'] += home_team_goals
        league_table_new.loc[league_table_new['team'] == home_team, 'goals_against'] += away_team_goals
        league_table_new.loc[league_table_new['team'] == away_team, 'goals_for'] += away_team_goals
        league_table_new.loc[league_table_new['team'] == away_team, 'goals_against'] += home_team_goals

        # update 'last_date' column:
        league_table_new.loc[league_table_new['team'] == home_team, 'last_date'] = last_date

        # update 'ponts' and 'w', 'd', 'l' columns based on the result of the match:
        if full_time_result == 'H':
            league_table_new.loc[league_table_new['team'] == home_team, 'points'] += 3
            league_table_new.loc[league_table_new['team'] == away_team, 'points'] += 0

            league_table_new.loc[league_table_new['team'] == home_team, 'w'] += 1
            league_table_new.loc[league_table_new['team'] == away_team, 'l'] += 1

        elif full_time_result == 'A':
            league_table_new.loc[league_table_new['team'] == home_team, 'points'] += 0
            league_table_new.loc[league_table_new['team'] == away_team, 'points'] += 3

            league_table_new.loc[league_table_new['team'] == home_team, 'l'] += 1
            league_table_new.loc[league_table_new['team'] == away_team, 'w'] += 1

        elif full_time_result == 'D':
            league_table_new.loc[league_table_new['team'] == home_team, 'points'] += 1
            league_table_new.loc[league_table_new['team'] == away_team, 'points'] += 1

            league_table_new.loc[league_table_new['team'] == home_team, 'd'] += 1
            league_table_new.loc[league_table_new['team'] == away_team, 'd'] += 1
        else:
            print('Error: FTR is not H, A or D')

        # update 'goal_difference' column:
        league_table_new['goal_difference'] = league_table_new['goals_for'] - league_table_new['goals_against']

        # crate ranking based on points, goal difference, goals for and goals against
        league_table_new = league_table_new.sort_values(by=['points', 'goal_difference', 'goals_for', 'goals_against'], ascending=False)
        league_table_new = league_table_new.reset_index(drop=True)
        league_table_new['ranking'] = league_table_new.index + 1

        # update our data
        updated_row = league_table_new[(league_table_new['team'] == home_team) | (league_table_new['team'] == away_team)]
        league_table_all = pd.concat([league_table_all, updated_row], ignore_index=True)

    # reorder rows based on column points and goals_for:
    league_table_new = league_table_new.sort_values(by=['points', 'goal_difference', 'goals_for', 'goals_against'], ascending=False)
    league_table_new = league_table_new.reset_index(drop=True)

    return league_table_new, league_table_all

In [11]:
new_table_data,  league_table_all= update_league_table(table_data, results_data)

In [12]:
new_table_data

,team,ranking,points,w,d,l,goals_for,goals_against,goal_difference,matched_played,last_date
0,Real Madrid,1,586,177,55,66,627,330,297,298,2023-05-09
1,Barcelona,2,563,166,65,52,564,282,282,283,2022-10-26
2,Bayern Munich,3,557,167,56,57,577,277,300,280,2023-04-19
3,Manchester United,4,460,133,61,54,429,232,197,248,2022-03-15
4,Juventus,5,389,112,53,53,351,218,133,218,2022-11-02
...,...,...,...,...,...,...,...,...,...,...,...
154,MŠK Žilina,155,0,0,0,4,2,11,-9,4,2010-09-15
155,Austria Wien,156,0,0,0,4,1,12,-11,4,2013-10-22
156,Levski Sofia,157,0,0,0,4,1,12,-11,4,2006-11-22
157,Sturm Graz,158,0,0,0,8,3,24,-21,8,2000-12-06


# Adding the ELO score

In [42]:
def create_league_table_elo(data, default_elo=1400, custom_elo=None):
    unique_team_names = data['HomeTeam'].unique()

    league_table = pd.DataFrame()

    league_table['team'] = unique_team_names
    league_table['ranking'] = 0

    # Set default Elo for all teams
    league_table['elo'] = default_elo

    # If custom Elo ratings are provided, update Elo for specific teams
    if custom_elo:
        for team, elo in custom_elo.items():
            if team in league_table['team'].values:
                league_table.loc[league_table['team'] == team, 'elo'] = elo
            else:
                print(f"Warning: {team} not found in data, Elo rating not updated.")

    # Remaining columns initialization
    league_table['points'] = 0
    league_table['w'] = 0
    league_table['d'] = 0
    league_table['l'] = 0
    league_table['goals_for'] = 0
    league_table['goals_against'] = 0
    league_table['goal_difference'] = 0
    league_table['matched_played'] = 0

    # Update 'last_date' column if 'Date' column exists in data
    if 'Date' in data.columns:
        league_table['last_date'] = min(data['Date'])
    else:
        league_table['last_date'] = 0

    return league_table

# List of teams with custom Elo ratings
custom_elo_ratings = {
    'Manchester City': 1500,
    'Liverpool': 1500,
    'Tottenham Hotspur': 1500,
    'Chelsea': 1500,
    'Manchester United': 1500,
    'Arsenal': 1500,
    'Paris Saint Germain': 1500,
    'Barcelona': 1500,
    'Real Madrid': 1700,
    'Atletico Madrid': 1500,
    'Internazionale': 1500,
    'Milan': 1500,
    'Juventus': 1500,
    'Dortmund': 1500
}

# Create league table with custom Elo ratings
league_table_elo = create_league_table_elo(results_data, default_elo=1400, custom_elo=custom_elo_ratings)


In [29]:
def expected_result_prob(home_score,away_score, weight = 300, home_field_advantage = 30):
  # calculate the expected probabilities for a match between two teams
  diff_h_a = home_score - away_score + home_field_advantage #here we can add home field advantage

  we = (1/(10 ** (-diff_h_a/weight)+1)) #the actual formula to calculate the expected probability

  home_team_prop = np.round(we,3)
  away_team_prob = 1-home_team_prop
  away_team_prob = np.round(away_team_prob,3)
  return home_team_prop, away_team_prob

In [30]:
def actual_result(result):
  if result == 'H':
      home_team=1
      away_team=0
  elif result == 'A':
      home_team=0
      away_team=1
  elif result == 'D':
      home_team=0.5
      away_team=0.5
  return home_team, away_team

In [31]:
def calculate_elo(elo_home, elo_away, final_result, k_value = 25, weight = 300, home_field_advantage = 30):

  k=k_value

  erh, era = expected_result_prob(elo_home,elo_away, weight = weight, home_field_advantage = home_field_advantage)
  arh, ara = actual_result(final_result)

  # the actual formula to calculate the new elo
  updated_elo_home=elo_home + k*(arh-erh)
  updated_elo_away=elo_away + k*(ara-era)

  # round to nearest integer
  updated_elo_home=np.round(updated_elo_home, 3)
  updated_elo_away=np.round(updated_elo_away, 3)

  # just to calculate the stakes
  win_home = (1 - erh) * k
  lose_home = (0 - erh) * k
  draw_home = (0.5 - erh) * k
  win_away = (1 - era) * k
  lose_away = (0 - era) * k
  draw_away = (0.5 - era) * k
  stakes = [win_home, lose_home, draw_home, win_away, lose_away, draw_away]

  return updated_elo_home, updated_elo_away, stakes

In [43]:
def update_league_table(league_table, results_table, k_value = 25, weight = 300, home_field_advantage = 30):

    league_table_all = league_table.copy()
    league_table_new = league_table.copy()

    for index, row in results_table.iterrows():
        # get team names
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        # get match results
        home_team_goals = row['FTHG']
        away_team_goals = row['FTAG']
        full_time_result = row['FTR']
        # get date of match
        last_date = row['Date']
        # get elo of teams
        home_team_elo = league_table_new[league_table_new['team'] == home_team]['elo'].values[0]
        away_team_elo = league_table_new[league_table_new['team'] == away_team]['elo'].values[0]


        # update 'matched_played' column:
        league_table_new.loc[league_table_new['team'] == home_team, 'matched_played'] += 1
        league_table_new.loc[league_table_new['team'] == away_team, 'matched_played'] += 1

        # update 'goals_for' and 'goals_against' columns:
        league_table_new.loc[league_table_new['team'] == home_team, 'goals_for'] += home_team_goals
        league_table_new.loc[league_table_new['team'] == home_team, 'goals_against'] += away_team_goals
        league_table_new.loc[league_table_new['team'] == away_team, 'goals_for'] += away_team_goals
        league_table_new.loc[league_table_new['team'] == away_team, 'goals_against'] += home_team_goals

        # update 'last_date' column:
        league_table_new.loc[league_table_new['team'] == home_team, 'last_date'] = last_date

        # update 'ponts' and 'w', 'd', 'l' columns based on the result of the match:
        if full_time_result == 'H':
            league_table_new.loc[league_table_new['team'] == home_team, 'points'] += 3
            league_table_new.loc[league_table_new['team'] == away_team, 'points'] += 0

            league_table_new.loc[league_table_new['team'] == home_team, 'w'] += 1
            league_table_new.loc[league_table_new['team'] == away_team, 'l'] += 1

        elif full_time_result == 'A':
            league_table_new.loc[league_table_new['team'] == home_team, 'points'] += 0
            league_table_new.loc[league_table_new['team'] == away_team, 'points'] += 3

            league_table_new.loc[league_table_new['team'] == home_team, 'l'] += 1
            league_table_new.loc[league_table_new['team'] == away_team, 'w'] += 1

        elif full_time_result == 'D':
            league_table_new.loc[league_table_new['team'] == home_team, 'points'] += 1
            league_table_new.loc[league_table_new['team'] == away_team, 'points'] += 1

            league_table_new.loc[league_table_new['team'] == home_team, 'd'] += 1
            league_table_new.loc[league_table_new['team'] == away_team, 'd'] += 1
        else:
            print('Error: FTR is not H, A or D')

        # update 'elo' score based on the result of the match:
        new_home_elo_score, new_away_elo_score, _ = calculate_elo(home_team_elo, away_team_elo, full_time_result, k_value = k_value, weight = weight, home_field_advantage = home_field_advantage)
        league_table_new.loc[league_table_new['team'] == home_team, 'elo'] = new_home_elo_score
        league_table_new.loc[league_table_new['team'] == away_team, 'elo'] = new_away_elo_score

        # update 'goal_difference' column:
        league_table_new['goal_difference'] = league_table_new['goals_for'] - league_table_new['goals_against']

        # crate ranking based on points, goal difference, goals for and goals against
        league_table_new = league_table_new.sort_values(by=['points', 'goal_difference', 'goals_for', 'goals_against'], ascending=False)
        league_table_new = league_table_new.reset_index(drop=True)
        league_table_new['ranking'] = league_table_new.index + 1

        # update our data
        updated_row = league_table_new[(league_table_new['team'] == home_team) | (league_table_new['team'] == away_team)]
        league_table_all = pd.concat([league_table_all, updated_row], ignore_index=True)

    # reorder rows based on column points and goals_for:
    league_table_new = league_table_new.sort_values(by=['points', 'goal_difference', 'goals_for', 'goals_against'], ascending=False)
    league_table_new = league_table_new.reset_index(drop=True)

    return league_table_new, league_table_all

In [44]:
league_table_elo, league_table_elo_all = update_league_table(league_table_elo, results_data, k_value = 25, weight = 300, home_field_advantage = 30)

In [45]:
league_table_elo = league_table_elo[league_table_elo['matched_played'] > 50]

In [46]:
league_table_elo.sort_values(by= 'elo', ascending= False)

,team,ranking,elo,points,w,d,l,goals_for,goals_against,goal_difference,matched_played,last_date
2,Bayern Munich,3,1678.375,557,167,56,57,577,277,300,280,2023-04-19
12,Manchester City,13,1644.050,212,63,23,26,238,125,113,112,2023-05-17
0,Real Madrid,1,1609.200,586,177,55,66,627,330,297,298,2023-05-09
11,Paris Saint Germain,12,1578.625,233,69,26,37,262,157,105,132,2023-02-14
7,Liverpool,8,1575.850,309,91,36,37,302,155,147,164,2023-02-21
1,Barcelona,2,1553.050,563,166,65,52,564,282,282,283,2022-10-26
5,Chelsea,6,1545.725,354,101,51,43,336,178,158,195,2023-04-18
9,Internazionale,10,1538.725,250,70,40,42,214,165,49,152,2023-05-16
6,Arsenal,7,1523.350,330,96,42,52,310,206,104,190,2017-03-07
4,Juventus,5,1505.375,389,112,53,53,351,218,133,218,2022-11-02


In [52]:
league_table_elo[['team', 'elo', 'w', 'd', 'l', 'goals_for',
       'goals_against', 'goal_difference', 'matched_played']].sort_values(by= 'elo', ascending= False)

,team,elo,w,d,l,goals_for,goals_against,goal_difference,matched_played
2,Bayern Munich,1678.375,167,56,57,577,277,300,280
12,Manchester City,1644.050,63,23,26,238,125,113,112
0,Real Madrid,1609.200,177,55,66,627,330,297,298
11,Paris Saint Germain,1578.625,69,26,37,262,157,105,132
7,Liverpool,1575.850,91,36,37,302,155,147,164
1,Barcelona,1553.050,166,65,52,564,282,282,283
5,Chelsea,1545.725,101,51,43,336,178,158,195
9,Internazionale,1538.725,70,40,42,214,165,49,152
6,Arsenal,1523.350,96,42,52,310,206,104,190
4,Juventus,1505.375,112,53,53,351,218,133,218


In [54]:
import pandas as pd
import numpy as np

# Define the functions for Elo calculation
def expected_result_prob(home_score, away_score, weight=400, home_field_advantage=50):
    diff_h_a = home_score - away_score + home_field_advantage
    we = 1 / (10 ** (-diff_h_a / weight) + 1)
    home_team_prop = np.round(we, 3)
    away_team_prob = 1 - home_team_prop
    away_team_prob = np.round(away_team_prob, 3)
    return home_team_prop, away_team_prob

def actual_result(result):
    if result == 'H':
        home_team = 1
        away_team = 0
    elif result == 'A':
        home_team = 0
        away_team = 1
    elif result == 'D':
        home_team = 0.5
        away_team = 0.5
    return home_team, away_team

def calculate_elo(elo_home, elo_away, final_result, k_value=40, weight=400, home_field_advantage=50):
    k = k_value
    erh, era = expected_result_prob(elo_home, elo_away, weight=weight, home_field_advantage=home_field_advantage)
    arh, ara = actual_result(final_result)
    updated_elo_home = elo_home + k * (arh - erh)
    updated_elo_away = elo_away + k * (ara - era)
    updated_elo_home = np.round(updated_elo_home, 3)
    updated_elo_away = np.round(updated_elo_away, 3)
    return updated_elo_home, updated_elo_away

# Load the dataset
df = results_data  # Replace "your_dataset.csv" with the actual filename

# Initialize Elo ratings for each team
elo_ratings = {}

# Iterate over each row in the dataset
for index, row in df.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    result = row['FTR']

    # Initialize Elo ratings for new teams
    if home_team not in elo_ratings:
        elo_ratings[home_team] = 1500
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1500

    # Calculate Elo ratings for the current match
    elo_home, elo_away = calculate_elo(elo_ratings[home_team], elo_ratings[away_team], result)

    # Update Elo ratings dictionary
    elo_ratings[home_team] = elo_home
    elo_ratings[away_team] = elo_away

    # Update DataFrame with Elo ratings
    df.at[index, 'home_elo'] = elo_home
    df.at[index, 'away_elo'] = elo_away


In [55]:
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,home_elo,away_elo
937,1993-09-15,Honvéd,Manchester United,2.0,3.0,A,1477.16,1522.84
2016,1993-09-15,Aarau,Milan,0.0,1.0,A,1477.16,1522.84
1605,1993-09-15,Dynamo Kyiv,Barcelona,3.0,1.0,H,1517.16,1482.84
938,1993-09-29,Manchester United,Honvéd,2.0,1.0,H,1537.48,1462.52
2017,1993-09-29,Milan,Aarau,0.0,0.0,D,1517.48,1482.52
...,...,...,...,...,...,...,...,...
1623,2023-04-19,Internazionale,Benfica,3.0,3.0,D,1651.96,1598.76
10,2023-05-09,Real Madrid,Manchester City,1.0,1.0,D,1791.36,1811.72
1624,2023-05-10,Milan,Internazionale,0.0,2.0,A,1566.52,1671.00
1625,2023-05-16,Internazionale,Milan,1.0,0.0,H,1682.64,1554.88


In [56]:
X = []
y = []


home_elo = df['home_elo']
away_elo = df['away_elo']

home_score = df['FTHG']
away_score = df['FTAG']


X.append(np.stack((home_elo, away_elo), axis=-1))
y.append(np.stack((home_score, away_score), axis=-1))


X = np.concatenate(X)
y = np.concatenate(y)


In [57]:
X.shape, y.shape

((2293, 2), (2293, 2))

In [58]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=2022
)

#PoissonRegressor works with a single output,
#so we train two of them (one for "home" score, one for "away" score)

model = MultiOutputRegressor(
    PoissonRegressor(fit_intercept=True)
)

# Fit our PoissonRegressor
model.fit(X_train, y_train)

# See the predicted deviance explained on the test set
model.score(X_test, y_test)

0.22677478969796716

In [59]:

# List of teams to filter
teams_to_keep = ['Manchester City', 'Liverpool', 'Tottenham Hotspur', 'Chelsea',
                 'Manchester United', 'Arsenal', 'Paris Saint Germain', 'Barcelona',
                 'Real Madrid', 'Atletico Madrid', 'Internazionale', 'Milan',
                 'Juventus', 'Bayern Munich', 'Dortmund','Ajax']

league_table_elo = league_table_elo[league_table_elo['team'].isin(teams_to_keep)]


In [60]:
league_table_elo=league_table_elo[['team', 'elo']]

In [61]:
league_table_elo.columns= ['Team', 'ELO']

In [62]:
name_list = league_table_elo['Team'].tolist()

In [63]:
name_list

['Real Madrid',
 'Barcelona',
 'Bayern Munich',
 'Manchester United',
 'Juventus',
 'Chelsea',
 'Arsenal',
 'Liverpool',
 'Milan',
 'Internazionale',
 'Dortmund',
 'Paris Saint Germain',
 'Manchester City',
 'Atletico Madrid',
 'Tottenham Hotspur',
 'Ajax']

In [64]:
# Convert columns to lists

value_list = league_table_elo['ELO'].tolist()

In [65]:
value_list

[1609.2,
 1553.05,
 1678.375,
 1485.175,
 1505.375,
 1545.725,
 1523.35,
 1575.85,
 1455.3,
 1538.725,
 1487.25,
 1578.625,
 1644.05,
 1475.225,
 1475.85,
 1456.575]

In [66]:
elo_init = dict(zip(name_list, value_list))
elo_init

{'Real Madrid': 1609.2,
 'Barcelona': 1553.05,
 'Bayern Munich': 1678.375,
 'Manchester United': 1485.175,
 'Juventus': 1505.375,
 'Chelsea': 1545.725,
 'Arsenal': 1523.35,
 'Liverpool': 1575.85,
 'Milan': 1455.3,
 'Internazionale': 1538.725,
 'Dortmund': 1487.25,
 'Paris Saint Germain': 1578.625,
 'Manchester City': 1644.05,
 'Atletico Madrid': 1475.225,
 'Tottenham Hotspur': 1475.85,
 'Ajax': 1456.575}

In [67]:
dict(sorted(elo_init.items(), key=lambda elo_init: elo_init[1], reverse=True))

{'Bayern Munich': 1678.375,
 'Manchester City': 1644.05,
 'Real Madrid': 1609.2,
 'Paris Saint Germain': 1578.625,
 'Liverpool': 1575.85,
 'Barcelona': 1553.05,
 'Chelsea': 1545.725,
 'Internazionale': 1538.725,
 'Arsenal': 1523.35,
 'Juventus': 1505.375,
 'Dortmund': 1487.25,
 'Manchester United': 1485.175,
 'Tottenham Hotspur': 1475.85,
 'Atletico Madrid': 1475.225,
 'Ajax': 1456.575,
 'Milan': 1455.3}

In [68]:
def simulate_matches(fixtures, teams_df=None, lam_multiplier=1):
    """Function to simulate matches.

    Parameters
    ----------
    fixtures : list (float or str), optional
        A 2D list of fixtures, either represented as strings when team_df is
        provided, or as ELO scores if not.

    team_df : Pandas dataframe, optional
        A Pandas dataframe with "Team" and "ELO" as columns. If fixtures is in
        str form, this will be used to map it to ELO float form.

    lam_multiplier : float, optional
        How much to scale a teams lambda value by. If they're expected to score
        2.5 goals in a game, and we want to know how many they'll score in a
        half, we can use lam_multiplier to scale the expected number of goals
        down to 1.25.

    Returns
    ------
    score : ndarray
        This is the same size as fixtures, with each element corresponding to
        how many goals each team scored in a simulation.
    """

    # If no fixtures to simulate, return none
    if(len(fixtures)) == 0:
        return None

    # If teams_df is provided, we need to map each teams name to their ELO
    if teams_df is not None:
        # Get ELO scores for each team
        elo_scores = np.vectorize(dict(zip(teams_df.Team, teams_df.ELO)).get)(fixtures).astype(np.float32)
    else:
        # Otherwise, we already have the ELO scores for each team
        elo_scores = fixtures

    # Predict expected number of goals per team
    lam = model.predict(elo_scores)

    # Using expected number of goals per team, draw the number of goals per team
    # from a Poisson distribution
    score = np.random.poisson(lam * lam_multiplier)

    return score

In [69]:
fixture_ELOs = [[2000, 1700], [1800, 1800]]

# Simulate these two matches
scores = simulate_matches(fixture_ELOs)

for ELOs, score in zip(fixture_ELOs, scores):
    print(f"ELOs: {ELOs}, score: {score}")

ELOs: [2000, 1700], score: [4 1]
ELOs: [1800, 1800], score: [0 1]


In [70]:
# Repeat the fixture between teams with ELOs of 2000 vs 1700 a total of 10,000 times
repeated_fixture = np.tile(np.array(fixture_ELOs)[:1], (10000, 1))

# Simulate this matche 10,000 times
scores = simulate_matches(repeated_fixture)

f"Team with an ELO of {fixture_ELOs[0][0]} wins {(scores[:, 0] > scores[:, 1]).mean() * 100:.1f}% of the time"

'Team with an ELO of 2000 wins 89.9% of the time'

In [71]:
def get_Ws(results, ELOs):
    """Returns result (1 for W, 0 for L, 0.5 for D) for the team with a higher ELO

    Parameters
    ----------
    results : ndarray (int)
        A 2D array of scores.

    ELOs : ndarray (float)
        A 2D array of ELOs for the teams in results.

    Returns
    ------
    Ws : ndarray
        The result for the "strongest" team in each fixture.
    """

    # Initialise Ws array
    Ws = np.zeros(ELOs.shape[:1])

    # Mask for games in which "home" and "away" teams won respectively
    team1_wins = results[:, 0] > results[:, 1]
    team2_wins = results[:, 0] < results[:, 1]

    # Mask for games that ended in draws
    draws = results[:, 0] == results[:, 1]

    # Matches with an "away" team that is stronger
    away_better = np.argmax(ELOs, axis=-1).astype(bool)

    # As above for "home"
    home_better = np.logical_not(away_better)

    # If "home" team won and they had a higher ELO, W is 1
    Ws[np.logical_and(home_better, team1_wins)] = 1

    # If "away" team won and they had a higher ELO, W is 1
    Ws[np.logical_and(away_better, team2_wins)] = 1

    # For draws, W is 1. Remaining values are already 0 so no need to change
    Ws[draws] = 0.5

    return Ws


def get_elo_adjustments(ELOs, Ws, results):
    """Get adjustment for ELOs of each team given the final score in each game.

    Parameters
    ----------
    results : ndarray (int)
        A 2D array of scores.

    ELOs : ndarray (float)
        A 2D array of ELOs for the teams in results.

    Ws : ndarray (float)
        A 1D array of results for the "strongest" team in each fixture.

    Returns
    ------
    ELO_changes_arr : ndarray
        Of the same shape as results, this says how much to adjust the ELO by.
        Each element corresponds to the same team in ELOs/results.
    """

    # Get difference in ELOs for each fixture
    drs = np.abs(ELOs[:, 0] - ELOs[:, 1])

    # Get probability of higher ELO team winning
    Ws_e = 1 / (10**(-drs/400) + 1)

    # Work out weight constant
    Ks = np.ones(ELOs.shape[0]) * 60

    score_diffs = np.abs(results[:, 0] - results[:, 1])
    Ks[np.where(score_diffs == 2)] *= 1.5
    Ks[np.where(score_diffs > 2)] *= 1 + (3/4 + (score_diffs[score_diffs > 2]-3)/8)

    # How much will ELOs change?
    ELO_changes = Ks * (Ws - Ws_e)

    # Put this into array showing changes for each team
    ELO_changes_arr = np.zeros((ELOs.shape[0], 2))
    ELO_changes_arr[np.arange(ELOs.shape[0]), np.argmax(ELOs, axis=1)] = ELO_changes
    ELO_changes_arr[np.arange(ELOs.shape[0]), 1 - np.argmax(ELOs, axis=1)] = -ELO_changes

    return ELO_changes_arr

In [72]:
import random

def fixtures(teams):
    if len(teams) % 2:
        teams.append('Day off')  # if team number is odd - use 'day off' as fake team

    # Create a set to keep track of matchups
    matchups = set()

    fixtures = []
    for i in range(0, len(teams) - 1):
        round_fixtures = []
        # Shuffle the teams to create random rotations
        random.shuffle(teams)
        for j in range(0, len(teams), 2):
            # Ensure that the matchup has not already occurred
            matchup = (teams[j], teams[j+1])
            reverse_matchup = (teams[j+1], teams[j])
            while matchup in matchups or reverse_matchup in matchups:
                random.shuffle(teams)
                matchup = (teams[j], teams[j+1])
                reverse_matchup = (teams[j+1], teams[j])
            round_fixtures.append(matchup)
            # Add the matchup and its reverse to the set
            matchups.add(matchup)
            matchups.add(reverse_matchup)
        fixtures.append(round_fixtures)

    return fixtures

# demo code
teams = ['Manchester City', 'Liverpool', 'Tottenham Hotspur', 'Chelsea',
         'Manchester United', 'Arsenal', 'Paris Saint Germain', 'Barcelona',
         'Real Madrid', 'Atletico Madrid', 'Internazionale', 'Milan',
         'Juventus', 'Bayern Munich', 'Dortmund', 'Ajax']

# for one match each - use this block only
matches = fixtures(teams)

matches


[[('Paris Saint Germain', 'Dortmund'),
  ('Barcelona', 'Chelsea'),
  ('Liverpool', 'Ajax'),
  ('Real Madrid', 'Arsenal'),
  ('Manchester City', 'Manchester United'),
  ('Atletico Madrid', 'Internazionale'),
  ('Milan', 'Juventus'),
  ('Bayern Munich', 'Tottenham Hotspur')],
 [('Manchester United', 'Milan'),
  ('Juventus', 'Tottenham Hotspur'),
  ('Internazionale', 'Arsenal'),
  ('Manchester City', 'Chelsea'),
  ('Atletico Madrid', 'Barcelona'),
  ('Bayern Munich', 'Ajax'),
  ('Liverpool', 'Real Madrid'),
  ('Dortmund', 'Liverpool')],
 [('Milan', 'Dortmund'),
  ('Manchester City', 'Bayern Munich'),
  ('Real Madrid', 'Barcelona'),
  ('Atletico Madrid', 'Paris Saint Germain'),
  ('Tottenham Hotspur', 'Liverpool'),
  ('Juventus', 'Manchester United'),
  ('Chelsea', 'Internazionale'),
  ('Arsenal', 'Ajax')],
 [('Manchester City', 'Juventus'),
  ('Real Madrid', 'Chelsea'),
  ('Manchester City', 'Liverpool'),
  ('Tottenham Hotspur', 'Chelsea'),
  ('Arsenal', 'Atletico Madrid'),
  ('Ajax', 'Pa

In [73]:
import pandas as pd

# Your list of tuples
fixtures_list = matches

# Initialize lists to store data
home_teams = []
away_teams = []
week_games = []

# Iterate over each week's fixtures
for week, fixtures in enumerate(fixtures_list, start=1):
    # Iterate over each game in the week
    for game, (home_team, away_team) in enumerate(fixtures, start=1):
        home_teams.append(home_team)
        away_teams.append(away_team)
        week_games.append(week)

# Create DataFrame
df = pd.DataFrame({'HomeTeam': home_teams, 'AwayTeam': away_teams, 'WeekGame': week_games})

# Display the DataFrame


In [74]:
df

,HomeTeam,AwayTeam,WeekGame
0,Paris Saint Germain,Dortmund,1
1,Barcelona,Chelsea,1
2,Liverpool,Ajax,1
3,Real Madrid,Arsenal,1
4,Manchester City,Manchester United,1
...,...,...,...
115,Arsenal,Milan,15
116,Bayern Munich,Barcelona,15
117,Atletico Madrid,Bayern Munich,15
118,Paris Saint Germain,Real Madrid,15


In [75]:
def simular_1fecha(all_groups_df, fixtures):
    """Esta función si que la cambie bastante.
    Simula los partidos de los fixtures que le pases (arriba defini arrays de fixtures)
    Y usa un df que se arma de 0 para ir metiendole la info ahí simulación a simulación.

    Parameters
    ----------
    all_groups_df : Dataframe
        A dataframe containing team names and ELOs.

    Returns
    ------
    sorted_groups_df : Dataframe
        Dataframe con la tabla despues de 1 fecha.
    """

    # Simulo con la lista de fixtures y el df creado.
    results = simulate_matches(fixtures, all_groups_df)

    # Get column indices for GF and GA
    GF_col = all_groups_df.columns.get_loc("GF")
    GA_col = all_groups_df.columns.get_loc("GA")

    # Update goals scored/conceded for "home" team
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[:, 0]), GF_col] += results[:, 0]
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[:, 1]), GA_col] += results[:, 0]
    # Update goals scored/conceded for "away" team
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[:, 1]), GF_col] += results[:, 1]
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[:, 0]), GA_col] += results[:, 1]

    # Index of ELO column
    ELO_col = all_groups_df.columns.get_loc("ELO")

    # Get ELO for "home" and "away" teams respectively
    team1_ELOs = np.array(all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[:, 0]), ELO_col])
    team2_ELOs = np.array(all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[:, 1]), ELO_col])

    # Combine into one ELO array
    ELOs = np.stack((team1_ELOs, team2_ELOs), axis=1).astype(np.int32)

    # Index of points column
    points_col = all_groups_df.columns.get_loc("Points")

    # Mask for games in which "home" team won
    team1_wins = results[:, 0] > results[:, 1]
    # Add 3 points for these teams
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[team1_wins, 0]), points_col] += 3
    # Mask for games in which "away" team won
    team2_wins = results[:, 0] < results[:, 1]
    # Add 3 points for these teams
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[team2_wins, 1]), points_col] += 3
    # Mask for games in which "away" team won
    draws = results[:, 0] == results[:, 1]
    # Add 1 point to each of these teams
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[draws].flatten()), points_col] += 1

    # Use get_Ws to get W value for ELO updating
    Ws = get_Ws(results, ELOs)

    # How much is to be added to ELO, elementwise matching teams in fixtures away
    elo_adjustments = get_elo_adjustments(ELOs, Ws, results)

    # Update ELO for "home" and "away" teams respectively
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[:, 0]), ELO_col] += elo_adjustments[:, 0]
    all_groups_df.iloc[pd.Index(all_groups_df.Team).get_indexer(fixtures[:, 1]), ELO_col] += elo_adjustments[:, 1]

    # Once all fixtures have taken place, add goal difference column
    all_groups_df["GD"] = all_groups_df["GF"] - all_groups_df["GA"]

    # Rank all teams on points, then GD/GF if tie, and group by Group
    sorted_groups_df = all_groups_df.sort_values(['Points', 'GD', 'GF'], ascending=[False, False, False]).reset_index(drop=True)

    return sorted_groups_df

In [78]:
teams = np.array(list(elo_init.keys()))

In [79]:
teams

array(['Real Madrid', 'Barcelona', 'Bayern Munich', 'Manchester United',
       'Juventus', 'Chelsea', 'Arsenal', 'Liverpool', 'Milan',
       'Internazionale', 'Dortmund', 'Paris Saint Germain',
       'Manchester City', 'Atletico Madrid', 'Tottenham Hotspur', 'Ajax'],
      dtype='<U19')

In [80]:
# Simulación entera de 1 torneo.

# Armo el df base
df_list = [[team, 0, 0, 0, elo_init[team]] \
            for i, team in enumerate(np.array(teams).flatten())]
all_groups_df = pd.DataFrame(df_list, columns=['Team', 'Points', 'GF', 'GA', 'ELO'])


def simular_torneo(df_torneo, all_groups_df):
    """
    Funcion de simular n fechas, necesito el df de cada fecha, los armo tipo fixutres y ahi mando la función de simular_1fecha.
    Cada df al final de la fecha ya a tener los elos ajustados entonces se va retroalimentando sola.
    """
    for x in range(1, 15):

        df_fecha = df[df['WeekGame'] == x ]
        fixtures = np.array(df_fecha[['HomeTeam', 'AwayTeam']])  # Corrected column name
        all_groups_df = simular_1fecha(all_groups_df, fixtures)
    return all_groups_df

df_test = simular_torneo(df, all_groups_df)


In [81]:
df_test

,Team,Points,GF,GA,ELO,GD
0,Manchester City,28,31,10,1745.070305,21
1,Dortmund,28,21,10,1653.414672,11
2,Arsenal,27,38,10,1743.643435,28
3,Real Madrid,25,22,12,1705.225634,10
4,Chelsea,24,18,15,1540.368919,3
5,Bayern Munich,21,26,17,1635.120483,9
6,Manchester United,21,15,10,1598.332159,5
7,Atletico Madrid,20,19,16,1515.037026,3
8,Paris Saint Germain,18,12,12,1532.449343,0
9,Internazionale,15,12,12,1515.599329,0


In [82]:
list_dfs_sim = []

nsimulations = 5000
for n in tqdm(range(0,nsimulations)):
    df_list = [[team, 0, 0, 0, elo_init[team]] \
            for i, team in enumerate(np.array(teams).flatten())]
    all_groups_df = pd.DataFrame(df_list, columns=['Team', 'Points', 'GF', 'GA', 'ELO'])
    df_simulacion_1_torneo = simular_torneo(df, all_groups_df)
    #Meto en la lista cada simulación terminada. Despues con esta lista voy a hacer magias para calcular las probs.
    list_dfs_sim.append(df_simulacion_1_torneo)

100%|██████████| 5000/5000 [15:23<00:00,  5.42it/s]


In [83]:
import pandas as pd
import numpy as np

# Assume list_dfs_sim contains simulation results
# Define a list of teams
teams = np.array(teams).flatten()  # Flatten teams if needed

# Initialize a dictionary to store counts of each team finishing first
team_counts = {team: 0 for team in teams}

# Count the number of times each team finishes first across all simulations
for df_sim in list_dfs_sim:
    # Get the team that finishes first in each simulation
    first_place_team = df_sim.iloc[0]['Team']  # Assuming the first row contains the winner
    # Increment the count for the team that finishes first
    team_counts[first_place_team] += 1

# Calculate the probability of each team winning the league
total_simulations = len(list_dfs_sim)
team_probabilities = {team: count / total_simulations for team, count in team_counts.items()}

# Sort the dictionary by probabilities in descending order
team_probabilities_sorted = dict(sorted(team_probabilities.items(), key=lambda item: item[1], reverse=True))

# Print the probabilities
for team, probability in team_probabilities_sorted.items():
    print(f"Team: {team}, Probability: {probability:.2%}")


Team: Manchester City, Probability: 29.98%
Team: Bayern Munich, Probability: 28.48%
Team: Real Madrid, Probability: 15.98%
Team: Liverpool, Probability: 5.40%
Team: Arsenal, Probability: 4.58%
Team: Internazionale, Probability: 3.80%
Team: Chelsea, Probability: 2.56%
Team: Paris Saint Germain, Probability: 2.36%
Team: Dortmund, Probability: 1.92%
Team: Atletico Madrid, Probability: 1.54%
Team: Barcelona, Probability: 1.18%
Team: Juventus, Probability: 0.70%
Team: Manchester United, Probability: 0.40%
Team: Milan, Probability: 0.40%
Team: Tottenham Hotspur, Probability: 0.36%
Team: Ajax, Probability: 0.36%
